<a href="https://colab.research.google.com/github/ddPn08/stable-diffusion-webui-colab/blob/main/sd-webui.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Stable Diffusion WebUI Colab by [ddPn08](https://github.com/ddpn08/)

This colab runs from the repo [`sd-webui`](https://github.com/sd-webui/stable-diffusion-webui)  
このColabは[`sd-webui`](https://github.com/sd-webui/stable-diffusion-webui) を使用しています。


## 1 -  Setup

### 1.1 Download repo and install

Clone git repo and setup miniconda
> Gitリポジトリをクローン、minicondaのセットアップ

In [ ]:
# @markdown ## Clone the stable-diffusion-webui repo

webui_branch = "master"  # @param {type: "string"}

! git clone https://github.com/sd-webui/stable-diffusion-webui /content/stable-diffusion
%cd /content/stable-diffusion
! git checkout {webui_branch}

import sys

! curl -O https://repo.anaconda.com/miniconda/Miniconda3-latest-Linux-x86_64.sh
! chmod +x Miniconda3-latest-Linux-x86_64.sh
! bash ./Miniconda3-latest-Linux-x86_64.sh -b -f -p /usr/local
sys.path.append("/usr/local/lib/python3.7/site-packages/")
! rm Miniconda3-latest-Linux-x86_64.sh

# @markdown Set up conda environment - Takes a whilp
# @markdown > conda環境をセットアップします (時間がかかります)
! conda env update -n base -f /content/stable-diffusion/environment.yaml

### 1.2 Setup GFPGAN and ESRGAN

In [ ]:
# @markdown **GFPGAN** Automatically correct distorted faces with a built-in GFPGAN option, fixes them in less than half a second
# @markdown > **GFPGAN** オプションで歪んだ顔を自動的に修正し、0.5秒未満で修正します

# @markdown **ESRGAN** Boosts the resolution of images with a built-in RealESRGAN option
# @markdown > **ESRGAN** オプションで画像の解像度を向上させます

add_CFP = True  # @param {type:"boolean"}
add_ESR = True  # @param {type:"boolean"}
add_LDSR = False  # @param {type:"boolean"}
# @markdown ⚠️ LDSR is 1.9GB and make take time to download
# @markdown > ※LDSRは1.9GBあり、ダウンロードに時間がかかります

if add_CFP:
    %cd /content/stable-diffusion/src/gfpgan/
    ! pip install basicsr facexlib yapf lmdb opencv-python pyyaml tb-nightly --no-deps
    ! python setup.py develop
    ! pip install realesrgan
    ! wget https://github.com/TencentARC/GFPGAN/releases/download/v1.3.0/GFPGANv1.3.pth -P experiments/pretrained_models
if add_ESR:
    %cd /content/stable-diffusion/src/realesrgan/
    ! wget https://github.com/xinntao/Real-ESRGAN/releases/download/v0.1.0/RealESRGAN_x4plus.pth -P experiments/pretrained_models
    ! wget https://github.com/xinntao/Real-ESRGAN/releases/download/v0.2.2.4/RealESRGAN_x4plus_anime_6B.pth -P experiments/pretrained_models
if add_LDSR:
    %cd /content/stable-diffusion/src
    ! git clone https://github.com/devilismyfriend/latent-diffusion
    %cd latent-diffusion
    %mkdir -p experiments/
    %cd experiments/
    %mkdir -p pretrained_models
    %cd pretrained_models
    # project.yaml download
    ! wget -O project.yaml https://heibox.uni-heidelberg.de/f/31a76b13ea27482981b4/?dl=1
    # model.ckpt model download
    ! wget -O model.ckpt https://heibox.uni-heidelberg.de/f/578df07c8fc04ffbadf3/?dl=1

%cd /content/stable-diffusion/
! wget https://github.com/matomo-org/travis-scripts/blob/master/fonts/Arial.ttf?raw=true -O arial.ttf

### 1.3 Setup model

In [ ]:
# @markdown # Load the stable-diffusion model
# @markdown > ### stable-diffusionのモデルをロード

# @markdown **Model Path Variables**
# ask for the link
print("Local Path Variables:\n")

model_filename = "sd-v1-4.ckpt"  # @param {type:"string"}
models_path = "/content/models"  # @param {type:"string"}
output_path = "/content/output"  # @param {type:"string"}

# @markdown **Download the model if it isn't already in the `models_path` folder (Optional)**
# @markdown > モデルが`models_path`フォルダーにない場合に、モデルをダウンロードする

# @markdown To download the model, you need to have accepted the terms [HERE](https://huggingface.co/CompVis/stable-diffusion-v1-4)
# @markdown and have copied a token from [HERE](https://huggingface.co/settings/tokens)

# @markdown > モデルをダウンロードするには, [このページ](https://huggingface.co/CompVis/stable-diffusion-v1-4)で条件に同意する必要があります。
# @markdown > また、[このページ](https://huggingface.co/settings/tokens)からトークンを取得する必要があります。
download_if_missing = False  # @param {type:"boolean"}
model_url = "https://huggingface.co/CompVis/stable-diffusion-v-1-4-original/resolve/main/sd-v1-4.ckpt"  # @param {type:"string"}
token = ""  # @param {type:"string"}

# @markdown **Google Drive Path Variables (Optional)**
mount_google_drive = True  # @param {type:"boolean"}
force_remount = False

%cd /content/
import os

mount_success = True
if mount_google_drive:
    from google.colab import drive

    try:
        drive_path = "/content/drive"
        drive.mount(drive_path, force_remount=force_remount)
        models_path_gdrive = "/content/drive/MyDrive/AI/models"  # @param {type:"string"}
        output_path_gdrive = "/content/drive/MyDrive/AI/sd-webui/outputs"  # @param {type:"string"}
        models_path = models_path_gdrive
        output_path = output_path_gdrive
    except:
        print("...error mounting drive or with drive path variables")
        print("...reverting to default path variables")
        mount_success = False

os.makedirs(models_path, exist_ok=True)
os.makedirs(output_path, exist_ok=True)

if download_if_missing:
    if not mount_success:
        print("Downloading model to " + models_path + " due to gdrive mount error")
    elif not os.path.exists(models_path + "/" + model_filename):
        ! mkdir sd-model
        %cd /content/sd-model/
        ! curl -LJ {model_url} -H {'"Authorization: Bearer ' + token + '"' if token else ""} -o {model_filename}
        ! mv /content/sd-model/{model_filename} {models_path}/
    else:
        print("Model already downloaded, moving to next step")

print(f"models_path: {models_path}")
print(f"output_path: {output_path}")

## 2 Optional - Set webUI settings and configs before running 
> オプション - 実行前にWebUIの設定を変更する。

In [ ]:
# @markdown # Launch preferences - Advanced
# @markdown # 詳細設定
share_password = ""  # @param {type:"string"}
# @markdown * Add a password to your webui
# @markdown > WebUI用のパスワード
defaults = "configs/webui/webui.yaml"  # @param {type:"string"}
# @markdown * path to configuration file providing UI defaults, uses same format as cli parameter)
# @markdown Edit this file if you want to change the default settings UI launches with
# @markdown > デフォルトUIの構成ファイルへのパス。
# @markdown UIのデフォルト設定を変更する場合は、このファイルを編集します

# @markdown ---
save_metadata = False  # @param {type:"boolean"}
# @markdown * Whether to embed the generation parameters in the sample images
# @markdown > サンプル画像に生成パラメータを埋め込む
skip_grid = False  # @param {type:"boolean"}
# @markdown * Do not save a grid, only individual samples. Helpful when evaluating lots of samples
# @markdown > グリッドは保存せず、個々のサンプルだけを保存する。多くのサンプルを評価する際に便利です
skip_save = False  # @param {type:"boolean"}
# @markdown * Do not save individual samples as files. For speed measurements
# @markdown > 個々のサンプルはファイルとして保存しない。速度測定用
optimized = False  # @param {type:"boolean"}
# @markdown * Load the model onto the device piecemeal instead of all at once to reduce VRAM usage at the cost of performance
# @markdown > パフォーマンスを犠牲にしてVRAMの使用量を減らすために、モデルを一度に全部ではなく断片的に読み込む
optimized_turbo = True  # @param {type:"boolean"}
# @markdown * Alternative optimization mode that does not save as much VRAM but runs siginificantly faster
# @markdown > VRAMの節約はできないが、動作が大幅に高速化される代替最適化モード
no_verify_input = False  # @param {type:"boolean"}
# @markdown * Do not verify input to check if it's too long
# @markdown > プロンプトが長すぎるかどうかを確認しない
no_half = False  # @param {type:"boolean"}
# @markdown * Do not switch the model to 16-bit floats
# @markdown > モデルを16ビットfloatに切り替えない
no_progressbar_hiding = True  # @param {type:"boolean"}
# @markdown * Do not hide progressbar in gradio UI
# @markdown > プログレスバーを非表示にしない
extra_models_cpu = False  # @param {type:"boolean"}
# @markdown * Run extra models (GFGPAN/ESRGAN) on cpu
# @markdown > 追加モデル(GFGPAN/ESRGAN)をCPUで実行する
esrgan_cpu = True  # @param {type:"boolean"}
# @markdown * run ESRGAN on cpu
# @markdown > ESRGANをCPUで実行する
gfpgan_cpu = False  # @param {type:"boolean"}
# @markdown * run GFPGAN on cpu
# @markdown > GFPGANをCPUで実行する


run_string_with_variables = {
    "--save-metadata": f"{save_metadata}",
    "--skip-grid": f"{skip_grid}",
    "--skip-save": f"{skip_save}",
    "--optimized": f"{optimized}",
    "--optimized-turbo": f"{optimized_turbo}",
    "--no-verify-input": f"{no_verify_input}",
    "--no-half": f"{no_half}",
    "--no-progressbar-hiding": f"{no_progressbar_hiding}",
    "--extra-models-cpu": f"{extra_models_cpu}",
    "--esrgan-cpu": f"{esrgan_cpu}",
    "--gfpgan-cpu": f"{gfpgan_cpu}",
}

only_true_vars = {k for (k, v) in run_string_with_variables.items() if v == "True"}
vars = " ".join(only_true_vars)

## 3 - Launch WebUI for stable diffusion

In [ ]:
#@markdown ** keep in mind that this script is set to run for ever, google will disconnect you after 90 minutes on free tiers
#@markdown > ** ※注意 このスクリプトは永久に実行されます。Google は無料枠で 90 分後に接続を切断します

#@markdown # Important - click the public URL to launch WebUI in another tab
#@markdown > ### 重要 - 公開URLをクリックしてWebUIを起動してください

#@markdown ![](https://user-images.githubusercontent.com/71378929/189563599-6df78bcf-133b-41e8-a55d-8ca3783cd933.png)

#fix adding share_password to the launch params, and also changin {vars} to $vars as it was causing webui.py to fail.

%cd /content/stable-diffusion

if share_password == "":
  ! python scripts/webui.py \
  --ckpt {models_path}/{model_filename} \
  --outdir {output_path} \
  --share $vars
else:
  ! python scripts/webui.py \
  --ckpt {models_path}/{model_filename} \
  --outdir {output_path} \
  --share-password {share_password} \
  --share $vars

